In [1]:
import pandas as pd 

In [ ]:
from nba_api.stats.endpoints import leaguegamelog, scoreboardv2  # Usamos ScoreboardV2 en lugar de V3
from nba_api.stats.static import teams
import pandas as pd

# Configuración
season = "2023-24"  # Cambia según la temporada
season_type = "Pre Season"  # Pretemporada

# --- 1. Obtener partidos ya jugados (LeagueGameLog) ---
try:
    gamelog = leaguegamelog.LeagueGameLog(season=season, season_type=season_type)
    games_played = gamelog.get_data_frames()[0]
    print(f"Partidos jugados en pretemporada: {len(games_played)}")
except Exception as e:
    print(f"No se pudieron obtener partidos jugados: {e}")
    games_played = pd.DataFrame()

# --- 2. Obtener próximos partidos (ScoreboardV2) ---
try:
    scoreboard = scoreboardv2.ScoreboardV2()  # Sin parámetros = partidos del día actual
    upcoming_games = scoreboard.line_score.get_data_frame()  # DataFrame con próximos partidos
    print(f"Próximos partidos: {len(upcoming_games)}")
except Exception as e:
    print(f"No se pudieron obtener próximos partidos: {e}")
    upcoming_games = pd.DataFrame()

# --- 3. Procesar y combinar datos ---
if not games_played.empty or not upcoming_games.empty:
    # Mapear team_id a nombres de equipos
    nba_teams = {team["id"]: team["full_name"] for team in teams.get_teams()}
    
    # Crear estructura compatible con tu tabla SQL
    final_data = []
    
    # Procesar partidos jugados (si existen)
    if not games_played.empty:
        for _, row in games_played.iterrows():
            home_team, visitor_team = row["MATCHUP"].split(" vs. ")  # Ejemplo: "MIA vs. BOS"
            game_info = {
                "game_id": row["GAME_ID"],
                "scheduled_date": pd.to_datetime(row["GAME_DATE"]).strftime("%Y-%m-%d %H:%M:%S"),
                "status": "scheduled",
                "home_team_id": row["TEAM_ID"],  # Equipo local (necesita ajuste)
                "visitor_team_id": visitor_team,  # Equipo visitante (necesita team_id)
                "stadium": "Unknown"
            }
            final_data.append(game_info)
    
    # Procesar próximos partidos (si existen)
    if not upcoming_games.empty:
        for _, row in upcoming_games.iterrows():
            game_info = {
                "game_id": row["GAME_ID"],
                "scheduled_date": pd.to_datetime(row["GAME_DATE_EST"]).strftime("%Y-%m-%d %H:%M:%S"),
                "status": "scheduled",
                "home_team_id": row["TEAM_ID"],
                "visitor_team_id": row["TEAM_ID_OPPOSING"],  # Necesita ajuste
                "stadium": "Unknown"
            }
            final_data.append(game_info)
    
    # Guardar en CSV
    df = pd.DataFrame(final_data)
    df.to_csv("nba_preseason_schedule.csv", index=False)
    print("✅ Datos guardados en 'nba_preseason_schedule.csv'")
else:
    print("❌ No se encontraron datos de pretemporada.")

ImportError: cannot import name 'scoreboardv3' from 'nba_api.stats.endpoints' (c:\Users\dnava\AppData\Local\Programs\Python\Python313\Lib\site-packages\nba_api\stats\endpoints\__init__.py)

In [3]:
games
